<a href="https://colab.research.google.com/github/SayatAbdikul/animal_classification/blob/main/VGG_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting the data

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from torch import nn

In [12]:
image_size = 224
transform = transforms.Compose([
    transforms.RandomResizedCrop(image_size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
folder_path = '/content/drive/My Drive/animals-10'
dataset = ImageFolder(root=folder_path, transform=transform)
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_data, val_data, test_data = random_split(dataset, [train_size, val_size, test_size])
class_names = dataset.classes

In [13]:
train_dataloader = DataLoader(train_data, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=32, shuffle=False)
test_dataloader = DataLoader(test_data, batch_size=32, shuffle=False)

In [14]:
import matplotlib.pyplot as plt
image = next(iter(train_dataloader))[0][0]
to_pil = transforms.ToPILImage()
plt.imshow(to_pil(image))
plt.show()

KeyboardInterrupt: 

# Helper functions

In [15]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc
def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> float:
    model.train()
    train_loss, train_acc = 0, 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        if len(X) == 0:
            print(f"empty batch on {batch}")
            continue
        y_pred = model(X)
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()
        train_acc += accuracy_fn(y_true=y, y_pred=y_pred.argmax(dim=1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch % 10 == 0:
          print(f"looked at batch: {batch}/ {len(dataloader)}")
    print(f"Train loss: {train_loss/len(train_dataloader):.5f} | Train accuracy: {train_acc/len(train_dataloader):.2f}%")
def evaluation_step(model: torch.nn.Module,
                    dataloader: torch.utils.data.DataLoader,
                    loss_fn: torch.nn.Module,
                    device: torch.device) -> float:
    model.eval()
    with torch.inference_mode():
        eval_loss, eval_acc = 0, 0
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)
            if len(X) == 0:
                print(f"empty batch on {batch}")
                continue
            y_pred = model(X)
            loss = loss_fn(y_pred, y)
            eval_loss += loss.item()
            eval_acc += accuracy_fn(y_true=y, y_pred=y_pred.argmax(dim=1))
        print(f"Model loss: {eval_loss/len(val_dataloader):.5f} | Model accuracy: {eval_acc/len(val_dataloader):.2f}%")

# Model classes

In [16]:
class ConvBlock(torch.nn.Module):
  def __init__(self, in_channels, out_channels, conv_number=2):
    super(ConvBlock, self).__init__()
    layers = []
    layers.append(torch.nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1))
    layers.append(torch.nn.ReLU())
    for i in range(conv_number-1):
      layers.append(torch.nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1))
      layers.append(torch.nn.ReLU())
    self.conv = torch.nn.Sequential(*layers)
  def forward(self, x):
    return self.conv(x)

In [17]:
class VGG16(torch.nn.Module):
  def __init__(self, in_channels=3, num_classes=10):
    super(VGG16, self).__init__()
    self.blocks = nn.Sequential(
        ConvBlock(in_channels, 64, 2),
        ConvBlock(64, 128, 2),
        ConvBlock(128, 256, 3),
        ConvBlock(256, 512, 3),
        ConvBlock(512, 512, 3)
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(512 * 7 * 7, 4096),
        nn.ReLU(),
        nn.Dropout(),
        nn.Linear(4096, 4096),
        nn.ReLU(),
        nn.Dropout(p=0.5),
        nn.Linear(4096, num_classes)
    )
  def forward(self, x):
    x = self.blocks(x)
    x = self.classifier(x)
    return x


# Model training/testing

In [18]:
model = VGG16(in_channels=3, num_classes=len(class_names))
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

VGG16(
  (blocks): Sequential(
    (0): ConvBlock(
      (conv): Sequential(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU()
        (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): ReLU()
      )
    )
    (1): ConvBlock(
      (conv): Sequential(
        (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU()
        (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): ReLU()
      )
    )
    (2): ConvBlock(
      (conv): Sequential(
        (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU()
        (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): ReLU()
        (4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (5): ReLU()
      )
    )
    (3): ConvBlock(
      (conv): Sequential(
        (0): Conv2d(256, 512, kernel_size=

In [ ]:
epochs = 3
for epoch in range(epochs):
  print(f"Epoch {epoch+1}/{epochs}")
  train_step(model, train_dataloader, loss_fn, optimizer, device)
  evaluation_step(model, val_dataloader, loss_fn, device)

Epoch 1/3
